In [78]:
from datetime import datetime
import glob
import numpy as np
import pathlib
import os
import pickle
import pandas as pd

from typing import Dict, List

from music21 import converter, instrument, stream, note, chord

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

# **Training**

### **PARAMETERS**

In [68]:
data_path = pathlib.Path('../raw_data/midi_chopin_format_0_dataset_full')
filenames = glob.glob(str(data_path/"*.mid"))
print('Number of files:', len(filenames))

Number of files: 139


In [69]:
##### PARAMETERS #####
SEQ_LENGTH = 25                 #default 100, sequence length
NUM_FILES = 1                 #len(filenames) for the whole dataset (number of files to use)
PERIOD = 1                      #default 10, saving weights every xx epoch
BATCH_SIZE = 16                 #default 64
EPOCHS = 1                     #default 200

NUM_NOTES_TO_GENERATE = 100     #default 500
STORAGE_PATH = '../raw_data/preprocessed_data'

### **FUNCTIONS**

In [70]:
def get_notes():
    """ Get all the notes and chords from the midi files - Call BEFORE train """
    notes = []
    
    print("Parsing files...")
    
    for file in filenames[:NUM_FILES]:
        stream_file = converter.parse(file)

        #print("Parsing %s" % file)
                
        components_to_parse = []
        for element in stream_file.recurse():
            components_to_parse.append(element)
        
        #components_to_parse = stream_file.flat.notes #return to this to preview model output 10 epochs 1st training
    
        for element in components_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(float(element.quarterLength)))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(float(element.quarterLength)))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(float(element.quarterLength)))

    with open(f"../raw_data/notes/notes_{NUM_FILES}files.pickle", 'wb') as f:
        pickle.dump(notes, f)
    
    print("Parsing done.")
    
    return notes

In [92]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = SEQ_LENGTH


    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = utils.to_categorical(network_output)
    
    #Transforming to pandas Dataframe and saving to CSV files 
    network_input_2d = np.reshape(network_input, (network_input.shape[0], -1))
    network_input_df = pd.DataFrame(network_input_2d)
    
    #network_output_2d = np.reshape(network_output, (network_output.shape[0], -1))
    network_output_df = pd.DataFrame(network_output)
    
    network_input_df.to_csv(STORAGE_PATH+'/network_inputs.csv',mode='w')
    network_output_df.to_csv(STORAGE_PATH+'/network_outputs.csv', mode='w')


    return (network_input, network_output)


In [72]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [73]:
def train(model, network_input, network_output):
    """ train the neural network """
    now = datetime.now()
    filepath = os.path.abspath(f"model_checkpoints_weights/{now.strftime('%d-%m/%H-%M-%S')}_weights-1LSTMAtt1LSTMLayer-num_files_{NUM_FILES}-seq_length_{SEQ_LENGTH}-batch_size_{BATCH_SIZE}"+"-{epoch:03d}-{loss:.4f}.hdf5")
    checkpoint = ModelCheckpoint(
        filepath,
        period=PERIOD, #Save weights every xx epochs
        monitor='loss',
        verbose=1,
        save_best_only=False,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)


In [74]:
def train_network(notes, n_vocab):
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

### **TRAIN MODEL**

In [75]:
#load files in
notes = get_notes()

# get amount of pitch names
n_vocab = len(set(notes))
print(f"n_vocab: {n_vocab}")

Parsing files...
Parsing done.
n_vocab: 1417


In [76]:
#check notes format
notes[:5]

['rest 0.25',
 'G#4 0.3333333333333333',
 'rest 2.4166666666666665',
 'B1 1.0',
 'rest 0.25']

In [93]:
#check network_input shape
network_input, network_output = prepare_sequences(notes, n_vocab)
network_input.shape

(5827, 25, 1)

In [85]:
network_output.shape

(5827, 1417)

In [12]:
#train model
train_network(notes, n_vocab) 

2022-12-12 09:51:58.036858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-12 09:51:58.037069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 09:51:58.037168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-12 09:51:58.037247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-12 09:51:58.037330: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

653/653 [==============================] - ETA: 0s - loss: 6.4118
Epoch 1: saving model to /home/chibidao/code/aeraymd/chopin_28_prelude/notebooks-vADHO/model_checkpoints_weights/12-12/09-51-58_weights-1LSTMAtt1LSTMLayer-num_files_3-seq_length_25-batch_size_16-001-6.4118.hdf5
653/653 [==============================] - 514s 778ms/step - loss: 6.4118


# **Generating music**

### **MANUAL INPUT BEFORE GENERATOR STEP**

In [35]:
#Path to the weights to load
WEIGHTS_PATH = 'model_checkpoints_weights/'+'12-12/09-51-58_weights-1LSTMAtt1LSTMLayer-num_files_3-seq_length_25-batch_size_16-001-6.4118.hdf5'
#Filename for generated midi file
OUTPUT_FILENAME = f"midi_output_{NUM_FILES}files"

### **FUNCTIONS**

In [36]:
def create_network_add_weights(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [37]:
def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = SEQ_LENGTH
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)


In [38]:
def generate_notes(model, network_input, pitchnames, n_vocab, NUM_NOTES_TO_GENERATE):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate notes
    for note_index in range(NUM_NOTES_TO_GENERATE):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [39]:
start = np.random.randint(0, len(network_input)-1)

# get all pitch names
pitchnames = sorted(set(item for item in notes))
pitchnames

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))


pattern = network_input[start]
prediction_output = []
network_input
# # generate notes
# for note_index in range(NUM_NOTES_TO_GENERATE):
#     prediction_input = np.reshape(pattern, (1, len(pattern), 1))
#     prediction_input = prediction_input / float(n_vocab)

#     prediction = model.predict(prediction_input, verbose=0)

#     index = np.argmax(prediction)
#     result = int_to_note[index]
#     prediction_output.append(result)

#     pattern.append(index)
#     pattern = pattern[1:len(pattern)]

array([[[0.98003992],
        [0.93263473],
        [0.99301397],
        ...,
        [0.98103792],
        [0.6501996 ],
        [0.97904192]],

       [[0.93263473],
        [0.99301397],
        [0.63772455],
        ...,
        [0.6501996 ],
        [0.97904192],
        [0.68313373]],

       [[0.99301397],
        [0.63772455],
        [0.98003992],
        ...,
        [0.97904192],
        [0.68313373],
        [0.99051896]],

       ...,

       [[0.27694611],
        [0.98453094],
        [0.98902196],
        ...,
        [0.98153693],
        [0.98003992],
        [0.08183633]],

       [[0.98453094],
        [0.98902196],
        [0.51047904],
        ...,
        [0.98003992],
        [0.08183633],
        [0.98153693]],

       [[0.98902196],
        [0.51047904],
        [0.98453094],
        ...,
        [0.08183633],
        [0.98153693],
        [0.51197605]]])

In [40]:
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [41]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            #new_rest = note.Rest(pattern)
            new_rest = note.Rest()
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)
    now = datetime.now()
    fp = '../raw_data/model_outputs/'+f"{now.strftime('%d-%m-%Y_%H-%M-%S')}_"+OUTPUT_FILENAME+'.mid'
    midifile = midi_stream.write('midi', fp=fp)
    #midi_stream.show('midi')
    return midifile


In [42]:
def generate():
    """ Generate a piano midi file """
    print("Generating notes...")
    #load the notes used to train the model
    notes = get_notes()
    with open(f"../raw_data/notes/notes_{NUM_FILES}files.pickle", 'rb') as f:
    #with open(f"../raw_data/notes/notes_reduced.pickle", 'rb') as f:    
        
        notes = pickle.load(f)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
    model = create_network_add_weights(normalized_input, n_vocab)
    model.build(input_shape=(None, normalized_input.shape[1], normalized_input.shape[2])) #e.g.(None,100,1)
    model.load_weights(WEIGHTS_PATH) # Load the weights to each node
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, NUM_NOTES_TO_GENERATE)
    midifile = create_midi(prediction_output)
    print("Notes generation done.")
    return midifile


In [43]:
# TEST

# Get all pitch names
#pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))
    
#print(pitchnames)
print(n_vocab)
#print(notes)
print(len(notes))


2004
10470


### **GENERATOR**

In [44]:
#RUN THE GENERATOR
midifile = generate()

Generating notes...
Parsing files...
Parsing done.
Notes generation done.


In [45]:
len(midifile)


68

In [46]:
Stream = converter.parse(midifile)

In [47]:
filenames[:NUM_FILES]

['../raw_data/midi_chopin_format_0_dataset_full/chopin_polonaise-fantasie_61_(c)lubetsky.mid',
 '../raw_data/midi_chopin_format_0_dataset_full/chpn-p2.mid',
 '../raw_data/midi_chopin_format_0_dataset_full/51126a_ballade_op_47_no_3_a_flat_(nc)smythe.mid']

In [48]:
# Listen to the model output
Stream.show('midi')

In [83]:
# Get the music score
Stream.show()

/home/chibidao/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/music21/musicxml/m21ToXml.py:510: MusicXMLWarning: <music21.stream.Score 0x7f0dfd56ca30> is not well-formed; see isWellFormedNotation()
  warnings.warn(f'{scOut} is not well-formed; see isWellFormedNotation()',


SubConverterException: Cannot find a path to the 'mscore' file at /usr/bin/mscore3 -- download MuseScore